# Gauss curvature approximation directly on surface

In [ ]:
from ngsolve import *
from netgen.meshing import MeshingStep
from netgen.csg import *
from netgen import meshing
from ngsolve.webgui import Draw
import random as random
from ngsolve.krylovspace import CGSolver

# Gauss curvature
def GaussCurv(mesh):
    nsurf = specialcf.normal(3)
    return Cof(Grad(nsurf))*nsurf*nsurf

# geodesic curvature
mu = Cross(specialcf.normal(3), specialcf.tangential(3))
edgecurve = specialcf.EdgeCurvature(3) # nabla_t t

# for angle deficit
bbndtang  = specialcf.VertexTangentialVectors(3)
bbndtang1 = bbndtang[:,0] 
bbndtang2 = bbndtang[:,1] 

In [ ]:
#H^-1 norm
def CompHm1Norm(rhs, order):
    fesH = H1(mesh, order=order)
    u,v = fesH.TnT()
    
    a = BilinearForm((Grad(u).Trace()*Grad(v).Trace()+u*v)*ds, symmetric=True, symmetric_storage=True, condense=True)
    f = LinearForm(rhs*v*ds).Assemble()
    
    apre = Preconditioner(a, "bddc")
    a.Assemble()
    invS = CGSolver(a.mat, apre.mat, printrates='\r', maxiter=400)
    ext = IdentityMatrix()+a.harmonic_extension
    inv = a.inner_solve + ext @ invS @ ext.T
    
    gfu = GridFunction(fesH)
    gfu.vec.data = inv*f.vec
    
    err = sqrt( Integrate( gfu**2, mesh, BND) + Integrate( Grad(gfu)**2, mesh, BND) )
    return err

In [ ]:
def ComputeCurv(mesh, order, Kex):
    fes = H1(mesh,order=order)
    u,v = fes.TnT()

    f = LinearForm(fes)
    f += GaussCurv(mesh)*v*ds
    f += -edgecurve*mu*v*ds(element_boundary=True)
    f += -v*acos(bbndtang1*bbndtang2)*ds(element_vb=BBND)


    M = BilinearForm(fes, symmetric=True, symmetric_storage=True, condense=True)
    M += u*v*ds

    gfK = GridFunction(fes)

    with TaskManager():
        f.Assemble()
        # angle deficit (no boundary)
        for i in range(mesh.nv):
            f.vec[i] += 2*pi
        
        Mpre = Preconditioner(M, "bddc")
        M.Assemble()
        invS = CGSolver(M.mat, Mpre.mat, printrates='\r', maxiter=400)
        ext = IdentityMatrix()+M.harmonic_extension
        inv = M.inner_solve + ext @ invS @ ext.T

        gfK.vec.data = inv*f.vec
        
        l2err = sqrt(Integrate((gfK-Kex)**2, mesh, BND))
        hm1err = CompHm1Norm(gfK-Kex,order=order+2)
        print("int gfK =", Integrate(gfK*ds(bonus_intorder=5),mesh))
        print("4*pi    =", 4*pi)
    Draw(gfK, mesh, "K")
    return l2err,hm1err,fes.ndof

In [ ]:
order=1
R = 2
geo = CSGeometry()
if True: #sphere
    geo.Add(Sphere(Pnt(0,0,0),R))
    Kex=1/R**2
else: # ellipsoid
    a = R
    b = R
    c = 3/4*R
    geo.Add(Ellipsoid(Pnt(0,0,0),Vec(a,0,0),Vec(0,b,0),Vec(0,0,c)))
    Kex = 1/(a**2*b**2*c**2*(x**2/a**4+y**2/b**4+z**2/c**4)**2)
errl2 = []
errhm1 = []
ndof = []

for i in range(4+(order==1)):
    mesh = Mesh(geo.GenerateMesh(maxh=0.5**i, perfstepsend=meshing.MeshingStep.MESHSURFACE))
    mesh.Curve(order)

    errl,errm1,dof = ComputeCurv(mesh, 2*order-1, Kex=Kex)
    errl2.append(errl)
    errhm1.append(errm1)
    ndof.append(dof)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(ndof, errhm1, '-o', label="$\|K_h-K_{\mathrm{ex}}\|_{H^{-1}}$")
plt.plot(ndof, errl2 , '-x', label="$\|K_h-K_{\mathrm{ex}}\|_{L^2}$")
plt.plot(ndof, [10**order*dof**(-(order-1)/2) for dof in ndof], '-.', color="k", label="$O(h^"+str(order-1)+")$")
plt.plot(ndof, [10**order*dof**(-(order)/2) for dof in ndof], '-' , color="k", label="$O(h^"+str(order)+")$")
plt.plot(ndof, [10**order*dof**(-(order+1)/2) for dof in ndof], '--', color="k", label="$O(h^"+str(order+1)+")$")
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.title("p="+str(order))
plt.show()